In [1]:
# tritonserver --model-repository=../models --model-control-mode=explicit


In [5]:
# Import dependencies
import os
from time import time

import numpy as np
import sys
import cudf



/usr/local/lib/python3.8/dist-packages/cudf/utils/metadata/orc_column_statistics_pb2.py:19: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
/usr/local/lib/python3.8/dist-packages/cudf/utils/metadata/orc_column_statistics_pb2.py:37: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
/usr/local/lib/python3.8/dist-packages/cudf/utils/metadata/orc_column_statistics_pb2.py:44: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.Fiel

In [6]:
import tritonhttpclient

try:
    triton_client = tritonhttpclient.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))
triton_client.is_server_live()



client created.
GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


True

In [7]:
triton_client.get_model_repository_index()

POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '167'}>
bytearray(b'[{"name":"t4r_pytorch","version":"1","state":"READY"},{"name":"t4r_pytorch_nvt","version":"1","state":"READY"},{"name":"t4r_pytorch_pt","version":"1","state":"READY"}]')


[{'name': 't4r_pytorch', 'version': '1', 'state': 'READY'},
 {'name': 't4r_pytorch_nvt', 'version': '1', 'state': 'READY'},
 {'name': 't4r_pytorch_pt', 'version': '1', 'state': 'READY'}]

In [8]:
model_name = "t4r_pytorch"
triton_client.load_model(model_name=model_name)



POST /v2/repository/models/t4r_pytorch/load, headers None
{}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 't4r_pytorch'


In [9]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "../data/fake_preprocessed_daily_input/")
df= cudf.read_parquet(os.path.join(INPUT_DATA_DIR, '2022-09-04.parquet'))
df=df.sort_values(['session_id', 'seconds'])
batch = df.iloc[:50,:]



In [10]:
sessions_to_use = batch.session_id.value_counts()
filtered_batch = batch[batch.session_id.isin(sessions_to_use[sessions_to_use.values>1].index.values)]

#pick one session
session_id = list(sessions_to_use.to_pandas().keys())[0]
print(f'Using session_id: {session_id}')
filtered_batch = batch[batch.session_id==session_id].reset_index(drop=True)
filtered_batch = filtered_batch[['session_id','seconds','pc9','price','product_item_type','color_group','action']]
filtered_batch.head()



Using session_id: 721544


,session_id,seconds,pc9,price,product_item_type,color_group,action
0,721544,1662319898,081563270,75.23,color_35,item_type_17,product_viewed
1,721544,1662319913,086563270,75.23,color_35,item_type_15,product_viewed
2,721544,1662320906,086563270,75.23,color_35,item_type_15,product_added
3,721544,1662320916,241563270,86.06,color_35,item_type_15,product_viewed
4,721544,1662320916,086563270,75.23,color_35,item_type_15,product_viewed


In [12]:

import warnings

warnings.filterwarnings("ignore")



In [13]:
import nvtabular.inference.triton as nvt_triton
import tritonclient.grpc as grpcclient

inputs = nvt_triton.convert_df_to_triton_input(filtered_batch.columns, filtered_batch, grpcclient.InferInput)

output_names = ["output"]

outputs = []
for col in output_names:
    outputs.append(grpcclient.InferRequestedOutput(col))
    
MODEL_NAME_NVT = "t4r_pytorch"

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_NVT, inputs)
    print(col, ':\n', response.as_numpy(col))



InferenceServerException: [StatusCode.INTERNAL] in ensemble 't4r_pytorch', Failed to process the request(s) for model instance 't4r_pytorch_nvt', message: IndexError: tuple index out of range

At:
  /usr/local/lib/python3.8/dist-packages/nvtabular/inference/workflow/pytorch.py(47): _transform_outputs
  /usr/local/lib/python3.8/dist-packages/nvtabular/inference/workflow/base.py(115): run_workflow
  /root/data/../models/t4r_pytorch_nvt/1/model.py(120): execute
